In [ ]:
!pip install lightning pytorch-metric-learning pillow albumentations timm
!pip install neptune

In [ ]:

  !wget -O train.zip "https://chmura.put.poznan.pl/s/Vc93ZH7TCtaWANk/download"
  !unzip -q train.zip

  !wget -O test.zip "https://chmura.put.poznan.pl/s/kJ6Q9tNpJ33osjr/download"
  !unzip -q test.zip

In [ ]:
# !unzip -q train.zip
# !unzip -q test.zip

In [11]:
from pathlib import Path
from typing import Callable

import numpy as np
import torch
from PIL import Image
from torch.utils.data import Dataset


class MetricLearningDataset(Dataset):
    def __init__(self,
                 places_dirs: list[Path],
                 number_of_places_per_batch: int,
                 number_of_images_per_place: int,
                 number_of_batches_per_epoch: int,
                 transforms: Callable):
        super().__init__()

        self._places_images_paths: list[list[Path]] = [
            sorted([image_path for image_path in place_dir.iterdir() if image_path.is_file()])
            for place_dir in places_dirs
        ]
        self._number_of_places = number_of_places_per_batch
        self._number_of_images_per_place = number_of_images_per_place
        self._number_of_samples_per_epoch = number_of_batches_per_epoch
        self._transforms = transforms

    def __len__(self) -> int:
        return self._number_of_samples_per_epoch

    def __getitem__(self, _: int) -> tuple[torch.Tensor, torch.Tensor]:
        selected_places_indices = torch.randperm(len(self._places_images_paths))[:self._number_of_places]
        transformed_images = []
        selected_images_place_ids = []
        for place_index in selected_places_indices:
            place_images = self._places_images_paths[place_index]
            selected_image_indices = torch.randperm(len(place_images))[:self._number_of_images_per_place]
            selected_images = [np.asarray(Image.open(place_images[image_index]))
                               for image_index in selected_image_indices]
            for image in selected_images:
                image = self._transforms(image=image)['image']

                transformed_images.append(image)
                selected_images_place_ids.append(place_index)

        transformed_images, selected_images_place_ids = self._shuffle(transformed_images, selected_images_place_ids)

        return torch.stack(transformed_images), torch.tensor(selected_images_place_ids)

    @staticmethod
    def _shuffle(images: list[torch.Tensor], place_ids: list[int]) -> tuple[list[torch.Tensor], list[int]]:
        indices = torch.randperm(len(images))

        return [images[index] for index in indices], [place_ids[index] for index in indices]

In [12]:
from pathlib import Path
from typing import Callable

import numpy as np
import torch
from PIL import Image
from torch.utils.data import Dataset


class EvaluationDataset(Dataset):
    def __init__(self,
                 places_dirs: list[Path],
                 number_of_images_per_place: int,
                 transforms: Callable,
                 return_indices: bool = False):
        super().__init__()

        self._places_images_paths: list[list[Path]] = [
            sorted([image_path for image_path in place_dir.iterdir()
                    if image_path.is_file()])[:number_of_images_per_place]
            for place_dir in places_dirs
        ]
        self._number_of_images_per_place = number_of_images_per_place
        self._transforms = transforms
        self._return_indices = return_indices

    def __len__(self) -> int:
        return len(self._places_images_paths) * self._number_of_images_per_place

    def __getitem__(self, index: int) -> tuple[torch.Tensor, torch.Tensor] | tuple[torch.Tensor, torch.Tensor, int, int]:
        place_index = index // self._number_of_images_per_place
        image_index = index % self._number_of_images_per_place
        image_path = self._places_images_paths[place_index][image_index]

        image = np.asarray(Image.open(image_path))
        image = self._transforms(image=image)['image']

        if self._return_indices:
            return image, torch.tensor(place_index), place_index, image_index

        return image, torch.tensor(place_index)

    def get_path_by_index(self, place_id: int, image_index: int) -> Path:
        return self._places_images_paths[place_id][image_index]

In [13]:
from pathlib import Path
from typing import Callable

import numpy as np
import torch
from PIL import Image
from torch.utils.data import Dataset


class PredictionDataset(Dataset):
    def __init__(self,
                 test_dir: Path,
                 transforms: Callable):
        super().__init__()

        self._images_paths: list[Path] = sorted([image_path for image_path in test_dir.iterdir()])
        self._transforms = transforms

    def __len__(self) -> int:
        return len(self._images_paths)

    def __getitem__(self, index: int) -> tuple[torch.Tensor, str]:
        image_path = self._images_paths[index]

        image = np.asarray(Image.open(image_path))
        image = self._transforms(image=image)['image']

        return image, image_path.stem

In [72]:
from pathlib import Path
from typing import Optional

import albumentations.pytorch
import timm.data
from lightning import pytorch as pl
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader


class MetricLearningDataModule(pl.LightningDataModule):
    def __init__(self, data_path: Path, number_of_places_per_batch: int, number_of_images_per_place: int,
                 number_of_batches_per_epoch: int, augment: bool, validation_batch_size: int, number_of_workers: int):
        super().__init__()

        self._data_path = Path(data_path)
        self._number_of_places_per_batch = number_of_places_per_batch
        self._number_of_images_per_place = number_of_images_per_place
        self._number_of_batches_per_epoch = number_of_batches_per_epoch
        self._validation_batch_size = validation_batch_size
        self._number_of_workers = number_of_workers

        self._transforms = albumentations.Compose([
            albumentations.CenterCrop(512, 512),
            albumentations.Normalize(timm.data.IMAGENET_DEFAULT_MEAN, timm.data.IMAGENET_DEFAULT_STD),
            albumentations.pytorch.transforms.ToTensorV2()
        ])
        self._augmentations = albumentations.Compose([
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.Rotate(limit=15, p=1.0),
            albumentations.Affine(scale=(0.9, 1.1), translate_percent=(-0.1, 0.1), p=1.0),
            albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1.0),
            albumentations.HueSaturationValue(p=0.5),
            albumentations.CenterCrop(512, 512),
            albumentations.Normalize(timm.data.IMAGENET_DEFAULT_MEAN, timm.data.IMAGENET_DEFAULT_STD),
            albumentations.CoarseDropout(max_holes=8, max_height=32, max_width=32, min_holes=1, min_height=8, min_width=8, fill_value=0),
            albumentations.pytorch.transforms.ToTensorV2()
        ]) if augment else self._transforms

        self.train_dataset = None
        self.val_dataset = None
        self.easy_test_dataset = None
        self.medium_test_dataset = None
        self.hard_test_dataset = None
        self.predict_dataset = None

    def get_places_dirs(self, data_dir: Path) -> list[Path]:
        return sorted(
            [place_dir for place_dir in data_dir.iterdir()
             if place_dir.is_dir() and len(list(place_dir.iterdir())) >= self._number_of_images_per_place]
        )

    def get_number_of_places(self, subset: str) -> int:
        assert subset in ['train', 'val', 'test']
        return len(self.get_places_dirs(self._data_path / subset))

    def setup(self, stage: Optional[str] = None):
        train_places_dirs = self.get_places_dirs(self._data_path / 'train')
        # TODO: validation dataset size can be changed
        train_places_dirs, val_places_dirs = train_test_split(train_places_dirs, test_size=0.2, random_state=42)

        print(f'Number of train places: {len(train_places_dirs)}')
        print(f'Number of val places: {len(val_places_dirs)}')

        self.train_dataset = MetricLearningDataset(
            train_places_dirs,
            self._number_of_places_per_batch,
            self._number_of_images_per_place,
            self._number_of_batches_per_epoch,
            self._augmentations,
        )
        self.val_dataset = EvaluationDataset(
            val_places_dirs,
            self._number_of_images_per_place,
            self._transforms,
        )
        self.predict_dataset = PredictionDataset(
            self._data_path / 'test',
            self._transforms
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset, batch_size=1, num_workers=self._number_of_workers,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset, batch_size=self._validation_batch_size, num_workers=self._number_of_workers,
        )

    def predict_dataloader(self):
        return DataLoader(
            self.predict_dataset, batch_size=self._validation_batch_size, num_workers=self._number_of_workers,
        )

In [73]:
import logging

import torch
from pytorch_metric_learning.distances import BaseDistance
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from pytorch_metric_learning.utils.inference import CustomKNN
from torchmetrics import Metric


class MultiMetric(Metric):
    def __init__(self, distance: BaseDistance):
        super().__init__()

        logger = logging.getLogger('PML')
        logger.setLevel(logging.WARN)

        knn = CustomKNN(distance, batch_size=256)
        self.calculator = AccuracyCalculator(include=('precision_at_1', 'mean_average_precision'), k=4,
                                             device=torch.device('cpu'),
                                             knn_func=knn)
        self.metric_names = self.calculator.get_curr_metrics()

        for metric_name in self.metric_names:
            self.add_state(metric_name, default=torch.tensor(0, dtype=torch.float32), dist_reduce_fx='sum')

        self.add_state('count', default=torch.tensor(0, dtype=torch.int64), dist_reduce_fx='sum')

    def update(self, vectors, labels):
        vectors = vectors.detach().cpu() if vectors.requires_grad else vectors.cpu()
        labels = labels.detach().cpu() if labels.requires_grad else labels.cpu()
        results = self.calculator.get_accuracy(vectors, labels, include=('precision_at_1', 'mean_average_precision'))
        for metric_name, metric_value in results.items():
            metric_state = getattr(self, metric_name)
            metric_state += metric_value

        self.count += 1

    def compute(self):
        return {
            metric_name: getattr(self, metric_name) / self.count for metric_name in self.metric_names
        }

In [74]:
import timm
import torch.linalg
from lightning import pytorch as pl
from pytorch_metric_learning import miners, losses, distances
from torchmetrics import MetricCollection


class EmbeddingModel(pl.LightningModule):
    def __init__(self,
                 embedding_size: int,
                 lr: float,
                 lr_patience: int):
        super().__init__()

        self.save_hyperparameters()

        self.lr = lr
        self.lr_patience = lr_patience

        self.network = timm.create_model('mobilenetv2_100', pretrained=True, num_classes=embedding_size)
        self.dropout = torch.nn.Dropout(p=0.2)
        # TODO: The distance, the miner and the loss function are subject to change
        # TODO: Adding embedding regularization is probably a good idea
        self.distance = distances.CosineSimilarity()
        self.miner = miners.MultiSimilarityMiner(distance=self.distance)
        self.loss_function = losses.TripletMarginLoss(distance=self.distance)

        self.val_outputs = None

        metrics = MetricCollection(MultiMetric(distance=self.distance))
        self.val_metrics = metrics.clone(prefix='val_')

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.network(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.squeeze(0)
        y = y.squeeze(0)
        y_pred = self.forward(x)
        loss = self.loss_function(y_pred, y, self.miner(y_pred, y))
        self.log('train_loss', loss, sync_dist=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx) -> None:
        x, y = batch
        x = x.squeeze(0)
        y = y.squeeze(0)
        y_pred = self.forward(x)
        self.val_outputs['preds'].append(y_pred.cpu())
        self.val_outputs['targets'].append(y.cpu())

    def predict_step(self, batch, batch_idx, **kwargs) -> tuple[torch.Tensor, list[str]]:
        x, y = batch
        x = x.squeeze(0)
        y_pred = self.forward(x)
        return y_pred.cpu(), y

    def on_validation_epoch_start(self) -> None:
        self.val_outputs = {
            'preds': [],
            'targets': [],
        }

    def on_validation_epoch_end(self) -> None:
        preds = torch.cat(self.val_outputs['preds'], dim=0)
        targets = torch.cat(self.val_outputs['targets'], dim=0)
        self.log_dict(self.val_metrics(preds, targets), sync_dist=True)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr,weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=self.lr_patience)
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_precision_at_1',
        }

In [75]:
import pickle
from pathlib import Path

import lightning.pytorch as pl

pl.seed_everything(42, workers=True)

# TODO: experiment with data module and model settings
datamodule = MetricLearningDataModule(
    data_path=Path('.'),
    number_of_places_per_batch=16,
    number_of_images_per_place=5,
    number_of_batches_per_epoch=100,
    augment=True,
    validation_batch_size=24,
    number_of_workers=6
)
model = EmbeddingModel(
    embedding_size=1024,
    lr=1e-3,
    lr_patience=10
)

model_summary_callback = pl.callbacks.ModelSummary(max_depth=-1)
checkpoint_callback = pl.callbacks.ModelCheckpoint(filename='{epoch}-{val_precision_at_1:.5f}', mode='max',
                                                    monitor='val_precision_at_1', verbose=True, save_last=True)
early_stop_callback = pl.callbacks.EarlyStopping(monitor='val_precision_at_1', mode='max', patience=50)
lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='epoch')
logger = pl.loggers.NeptuneLogger(project="aadamczak/ZPO",
    api_token="") #Neptune api
trainer = pl.Trainer(logger=logger,
    callbacks=[model_summary_callback, checkpoint_callback, early_stop_callback, lr_monitor],
    accelerator='gpu',
    max_epochs=50
)

trainer.fit(model=model, datamodule=datamodule)

Seed set to 42
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


https://app.neptune.ai/aadamczak/ZPO/e/ZPO-87


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

    | Name                          | Type                 | Params
-------------------------------------------------------------------------
0   | network                       | ResNet               | 21.8 M
1   | network.conv1                 | Conv2d               | 9.4 K 
2   | network.bn1                   | BatchNorm2d          | 128   
3   | network.act1                  | ReLU                 | 0     
4   | network.maxpool               | MaxPool2d            | 0     
5   | network.layer1                | Sequential           | 221 K 
6   | network.layer1.0              | BasicBlock           | 74.0 K
7   | network.layer1.0.conv1        | Conv2d               | 36.9 K
8   | network.layer1.0.bn1          | BatchNorm2d          | 128   
9   | network.layer1.0.drop_block   | Identity             | 0     
10  | network.layer1.0.act1         | ReLU                 | 0     
11  | network.layer1.0.aa           | Identity             | 0     

Number of train places: 800
Number of val places: 200


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Epoch 17, global step 1800: 'val_precision_at_1' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 18, global step 1900: 'val_precision_at_1' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 19, global step 2000: 'val_precision_at_1' was not in top 1
`Trainer.fit` stopped: `max_epochs=20` reached.


In [69]:
predictions = trainer.predict(model=model, ckpt_path=checkpoint_callback.best_model_path, datamodule=datamodule)

results = {}
for prediction in predictions:
    for embedding, identifier in zip(*prediction):
        results[identifier] = embedding.tolist()

with open('results.pickle', 'wb') as file:
    pickle.dump(results, file)

Restoring states from the checkpoint path at /.neptune/Untitled/ZPO-86/checkpoints/epoch=5-val_precision_at_1=0.83625.ckpt


Number of train places: 800
Number of val places: 200


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /.neptune/Untitled/ZPO-86/checkpoints/epoch=5-val_precision_at_1=0.83625.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

In [70]:
import requests


def main():
    student_id = 144610 # TODO: put your student id here
    distance_name = 'cosine'  # supported values are: manhattan, euclidean, cosine
    with open('results.pickle', 'rb') as file:
        predictions = file.read()

    response = requests.post(f'https://zpo.dpieczynski.pl/{student_id}',
                             headers={'distance': distance_name},
                             data=predictions)
    print(response.json())


if __name__ == '__main__':
    main()

{'rank_1_accuracy': 0.5724}
